In [237]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize
from statistics import mean

np.random.seed(5824)

In [238]:
r = 0.002
T = 12000 #in months
M = 120

# Data Generating

In [239]:
def configurations(code):
    if code == 1:
        d = 10
        
        B = np.random.uniform(0.5,1.5,d-1).reshape(1,(d-1))

        sigma = np.random.uniform(0.1/np.sqrt(12),0.3/np.sqrt(12),d-1)
        D = np.diag(sigma**2)

        alpha = np.zeros(d-1)
        alpha_M = np.tile(alpha,(T,1))

        R_factor_l = np.random.normal(0.009,np.sqrt(0.002),T).reshape((T, 1))
        mean = np.zeros(d-1)
        err = np.random.multivariate_normal(mean,D,size = T)

        R_1 = r + alpha_M +(R_factor_l-r)@B+err
        R = np.append(R_1,R_factor_l,axis=1)
    elif code == 2:
        d = 50

        B = np.random.uniform(0.5,1.5,d-1).reshape(1,(d-1))

        sigma = np.random.uniform(0.1/np.sqrt(12),0.3/np.sqrt(12),d-1)
        D = np.diag(sigma**2)

        alpha = np.zeros(d-1)
        alpha_M = np.tile(alpha,(T,1))

        R_factor_2 = np.random.normal(0.009,np.sqrt(0.002),T).reshape((T, 1))
        mean = np.zeros(d-1)
        err = np.random.multivariate_normal(mean,D,size = T)

        R_2 = r + alpha_M +(R_factor_2-r)@B+err
        R = np.append(R_2,R_factor_2,axis=1)
    elif code == 3:
        d = 10

        B = np.random.uniform(0.5,1.5,d-1).reshape(1,(d-1))

        sigma = np.random.uniform(0.1/np.sqrt(12),0.3/np.sqrt(12),d-1)
        D = np.diag(sigma**2)

        alpha = np.random.uniform(-0.05,0.05,d-1)
        alpha_M = np.tile(alpha,(T,1))

        R_factor_3 = np.random.normal(0.009,np.sqrt(0.002),T).reshape((T, 1))
        mean = np.zeros(d-1)
        err = np.random.multivariate_normal(mean,D,size = T)

        R_3 = r + alpha_M +(R_factor_3-r)@B+err
        R = np.append(R_3,R_factor_3,axis=1)
        
    elif code == 4:
        d = 10

        B = np.random.uniform(0.5,1.5,d-1).reshape(1,(d-1))

        sigma = np.random.uniform(0.1/np.sqrt(12),0.3/np.sqrt(12),d-1)
        D = np.diag(sigma**2)

        alpha = np.zeros(d-1)
        alpha_M = np.tile(alpha,(T,1))

        R_factor_4 = np.zeros(T)
        R_factor_4[0] = np.random.normal(0.009, 0.002)
        for t in range(1, T):
            R_factor_4[t] = 0.01 - 0.112*R_factor_4[t-1] + np.random.normal(0,0.002)
        R_factor_4=np.array(R_factor_4).reshape((T, 1))

        mean = np.zeros(d-1)
        err = np.random.multivariate_normal(mean,D,size = T)

        R_4 = r + alpha_M +(R_factor_4-r)@B+err
        R = np.append(R_4,R_factor_4,axis=1)
    return (d,B,D,alpha,R)

# Methods Implemented

In [240]:
target = 0.2/12

In [241]:
def allocation(d,B,D,alpha,R,method):
    #mu = np.mean(R, axis=0).reshape((d,1))
    if method == "theory":
        alpha = alpha.reshape(1,d-1)
        mu = np.append(0.009,r + alpha + B*(0.009-r)).reshape(d,1)
        sigma = np.block([[0.002, 0.002*B], [0.002*B.T, 0.002*B.T @ B + D]])
            
        e = np.ones(R.shape[1]).reshape(-1, 1)
        Sigma_inv = np.linalg.inv(sigma)
        aa = (target-r)/float((mu-r*e).T@Sigma_inv@(mu-r*e)) *Sigma_inv@(mu-r*e)
            
    elif method == "ew":
        a = np.ones(d)
        aa = a/d
        
    elif method == "mkt":
        a = np.zeros(d-1)
        aa = np.append(a,1)
        
    elif method == "mv":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)

        e = np.ones(R.shape[1]).reshape(-1, 1)
        Sigma_inv = np.linalg.inv(sigma)
        aa = (target-r)/float((mu-r*e).T@Sigma_inv@(mu-r*e)) *Sigma_inv@(mu-r*e)
        
    elif method == "1f":
        mu = np.mean(R, axis=0).reshape((d,1))
        B = []
        D = []
        X = R[-1]-r
        X=sm.add_constant(X)
        for i in range(d):
            Y = R[i]-r
            model = sm.OLS(Y, X).fit()
            beta = model.params[1]
            B = np.append(B,beta)
        B = B.reshape(d, 1)
        d = Y - model.predict(X)
        D = np.diag(d)
        V = X @ X.T
        sigma = B.T @ V @ B + D 
        
        e = np.ones(R.shape[1]).reshape(-1, 1)
        Sigma_inv = np.linalg.inv(sigma)
        aa = (target-r)/float((mu-r*e).T@Sigma_inv@(mu-r*e)) *Sigma_inv@(mu-r*e)
        
    elif method == "bs":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        Sigma_inv = np.linalg.inv(sigma)
        e = np.ones(R.shape[1]).reshape(-1, 1)

        mu_0 = (mu.T @ Sigma_inv @ e)/(e.T @ Sigma_inv @e) * e

        nu2 = (mu-mu_0).T@Sigma_inv@(mu-mu_0)
        alpha = (d+2)/(d+2+(M-d-2)*nu2)
        mu = alpha*mu_0 + (1-alpha)*mu

        aa = (target-r)/float((mu-r*e).T@Sigma_inv@(mu-r*e)) *Sigma_inv@(mu-r*e)
        
    elif method == "lw":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        
        C = (np.trace(sigma)/len(sigma))*np.identity(len(sigma))
        delta = np.linalg.norm(sigma-C,'fro')**2/len(sigma)
        beta = min(delta,np.sum([np.linalg.norm(np.matrix(R[i]-mu).T@np.matrix(R[i]-mu)-sigma,'fro')**2 for i in range(M)])/((len(R)**2)*len(sigma)))
        gamma = beta/delta
        
        cov = gamma*C+(1-gamma)*sigma
        
        e = np.ones(R.shape[1]).reshape(-1, 1)
        Sigma_inv = np.linalg.inv(cov)
        aa = (target-r)/float((mu-r*e).T@Sigma_inv@(mu-r*e)) *Sigma_inv@(mu-r*e)

    elif method == "min":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        e = np.ones(R.shape[1]).reshape(-1, 1)
        Sigma_inv = np.linalg.inv(sigma)
        aa = (Sigma_inv@e)/(e.T@Sigma_inv@e)
        
    elif method == "mv-c":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        aa = constrainedMV(d,sigma,mu)
        
    elif method == "bs-c":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        Sigma_inv = np.linalg.inv(sigma)
        e = np.ones(R.shape[1]).reshape(-1, 1)

        mu_0 = (mu.T @ Sigma_inv @ e)/(e.T @ Sigma_inv @e) * e

        nu2 = (mu-mu_0).T@Sigma_inv@(mu-mu_0)
        alpha = (d+2)/(d+2+(M-d-2)*nu2)
        mu = alpha*mu_0 + (1-alpha)*mu
        aa = constrainedMV(d,sigma,mu)
        
    elif method == "lw-c":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        
        C = (np.trace(sigma)/len(sigma))*np.identity(len(sigma))
        delta = np.linalg.norm(sigma-C,'fro')**2/len(sigma)
        beta = min(delta,np.sum([np.linalg.norm(np.matrix(R[i]-mu).T@np.matrix(R[i]-mu)-sigma,'fro')**2 for i in range(M)])/((len(R)**2)*len(sigma)))
        gamma = beta/delta
        
        cov = gamma*C+(1-gamma)*sigma
        
        aa = constrainedMV(d,cov,mu)
        
    elif method == "min-c":
        mu = np.mean(R, axis=0).reshape((d,1))
        sigma = np.cov(R.T)
        aa = minC(d,sigma,mu)
        
    return aa

In [242]:
def constrainedMV(d,sigma,mu):
    initial_weight = np.abs(np.random.randn(d))
    x0 = initial_weight/np.sum(initial_weight)

    cons = ({'type': 'ineq', 'fun': lambda x: r + np.dot(x,mu-r) - target },
            {'type': 'ineq', 'fun': lambda x:x },
            {'type': 'ineq', 'fun': lambda x: 1-np.sum(x)})

    res = minimize(func, x0,args = (sigma),constraints=cons )
    return np.array(res.x)

def func(x,sigma):
    return np.dot(np.dot(x,sigma),x)

def minC(d,sigma,mu):
    initial_weight = np.abs(np.random.randn(d))
    x0 = initial_weight/np.sum(initial_weight)

    cons = ({'type': 'ineq', 'fun': lambda x:x },
            {'type': 'ineq', 'fun': lambda x: 1-np.sum(x)})
    

    res = minimize(func, x0,args = (sigma),constraints=cons )
    return np.array(res.x)

In [243]:
def metrics_comp(d,B,D,alpha,R,M,method):
    sum_r = 0
    ret = []
    for t in range(M,T):
        ret_matrix = R[t-M:t]
        a = allocation(d,B,D,alpha,ret_matrix,method)
        Xt = R[t] @ a +(1-sum(a))*r
        ret.append(Xt)
        sum_r += Xt
    mu_OOS = 1/(T-M)*sum_r
    sd_x = np.std(ret)
    OSR = (mu_OOS-r)/sd_x
    
    return (mu_OOS,OSR)
    

In [247]:
methods = ['theory','ew','mkt','mv','1f','bs','lw','min','mv-c','bs-c','lw-c','min-c']
#methods = ['theory','ew','mkt','mv']
code = 1
d,B,D,alpha,R = configurations(code)
print('Configuration',code)
for method in methods:
    print(method)
    res = metrics_comp(d,B,D,alpha,R,M,method)
    print(res)

Configuration 1
theory
(array([0.01871908]), array([0.08428114]))
ew
(0.008835852147888125, 0.14977032247310548)
mkt
(0.0093216260679234, 0.16378152760655212)
mv
(array([0.00551467]), array([0.07002765]))
1f
(array([0.30755823]), array([0.01313289]))
bs
(array([0.00936927]), array([0.07006319]))
lw
(array([0.00999892]), array([0.1189111]))
min
(array([0.00752206]), array([0.13322838]))
mv-c
(0.009011397262110294, 0.10512266676086506)
bs-c
(0.010036266722349045, 0.10055726309535615)
lw-c
(0.009579258929975453, 0.11172483787563231)
min-c
(0.0021889000926298328, 0.08045301086982114)


In [248]:
code = 2
d,B,D,alpha,R = configurations(code)
print('Configuration',code)
for method in methods:
    print(method)
    res = metrics_comp(d,B,D,alpha,R,M,method)
    print(res)

Configuration 2
theory
(array([0.01787781]), array([0.11094425]))
ew
(0.009385942050200708, 0.15903383682529534)
mkt
(0.009369759831932983, 0.16250178225550244)
mv
(array([0.00275799]), array([0.02571023]))
1f
(array([0.11719499]), array([0.01285456]))
bs
(array([0.00351702]), array([0.02508805]))
lw
(array([0.01032776]), array([0.14102778]))
min
(array([0.00319359]), array([0.04438315]))
mv-c
(0.007893924308027676, 0.11260845924619177)
bs-c
(0.010524179084974815, 0.09791187902152564)
lw-c
(0.009981616065103194, 0.13643557615211444)
min-c
(0.0022207726095996856, 0.11452789128645197)


In [249]:
code = 3
d,B,D,alpha,R = configurations(code)
print('Configuration',code)
for method in methods:
    print(method)
    res = metrics_comp(d,B,D,alpha,R,M,method)
    print(res)

Configuration 3
theory
(array([0.00626172]), array([0.59916658]))
ew
(0.008650523935146315, 0.14615777004643188)
mkt
(0.00906926504042808, 0.1576290090479835)
mv
(array([0.01645161]), array([2.10093998]))
1f
(array([0.00685027]), array([0.00267702]))
bs
(array([0.01675133]), array([2.09996582]))
lw
(array([0.01619063]), array([1.5328906]))
min
(array([0.00920868]), array([0.19811816]))
mv-c
(0.01650835411035886, 0.894513555459499)
bs-c
(0.016708452072872908, 0.8906970057005301)
lw-c
(0.01650422423543491, 0.7055077856632719)
min-c
(0.0022138134022185504, 0.06627909961077877)


In [250]:
code = 4
d,B,D,alpha,R = configurations(code)
print('Configuration',code)
for method in methods:
    print(method)
    res = metrics_comp(d,B,D,alpha,R,M,method)
    print(res)

Configuration 4
theory
(array([0.00996967]), array([0.04565597]))
ew
(0.009575831438445079, 0.428527378293557)
mkt
(0.008986841725166329, 3.467404529245466)
mv
(array([0.01657926]), array([3.31149836]))
1f
(array([0.05770518]), array([0.01063849]))
bs
(array([0.01665011]), array([3.31794069]))
lw
(array([0.01273668]), array([0.3661577]))
min
(array([0.00897785]), array([3.33102528]))
mv-c
(0.010409682333580665, 0.18246688840480044)
bs-c
(0.010465237627178492, 0.13472653209562224)
lw-c
(0.010144820746391966, 0.15179945749325327)
min-c
(0.005067787367447477, 0.48990833265056327)
